In [1]:
import zipfile
from pathlib import Path

import pandas as pd


BASE_DIR = Path("/home/onyxia/france-grid-stress-prediction")
RAW_CONS_DIR = BASE_DIR / "data" / "raw*" / "consommation"
PROCESSED_DIR = BASE_DIR / "data" / "processed"

PROCESSED_DIR.mkdir(parents=True, exist_ok=True)


def load_year_raw(year: int) -> pd.DataFrame:
    """
    Ouvre le zip de consommation pour une année donnée et renvoie le DataFrame brut
    (XLS ou CSV, automatiquement détecté).
    """
    zip_path = RAW_CONS_DIR / f"Historique_consommation_INST_{year}.zip"

    if not zip_path.exists():
        print(f"[{year}] Fichier zip introuvable : {zip_path}")
        return None

    with zipfile.ZipFile(zip_path, "r") as z:
        inner = z.namelist()[0]  # on suppose un fichier principal dedans
        print(f"[{year}] Contenu du zip : {inner}")

        with z.open(inner) as f:
            if inner.lower().endswith(".csv"):
                df_raw = pd.read_csv(f, sep=";", engine="python")
            elif inner.lower().endswith((".xls", ".xlsx")):
                df_raw = pd.read_excel(f)
            else:
                print(f"[{year}] Format non supporté : {inner}")
                return None

    return df_raw


def tidy_year(df_raw: pd.DataFrame, year: int) -> pd.DataFrame:
    """
    Transforme le format 'large' journalier RTE en format 'long' :
    une ligne = un pas de temps (hypothèse : 30 minutes).
    Colonnes finales : datetime, date, statut, slot_index, load_mw, year.
    """
    # Renommer les colonnes en col_0, col_1, ...
    df = df_raw.copy()
    df.columns = [f"col_{i}" for i in range(df.shape[1])]

    # col_0 contient les dates au format 'dd/mm/YYYY' (ou NaN / texte hors données)
    dates = pd.to_datetime(df["col_0"], format="%d/%m/%Y", errors="coerce")
    df["date"] = dates

    # garder uniquement les lignes de données (date non nulle)
    df = df[df["date"].notna()].copy()

    # col_1 = statut (ex : 'Définitives')
    if "col_1" in df.columns:
        df = df.rename(columns={"col_1": "statut"})
    else:
        df["statut"] = None

    # supprimer les colonnes complètement vides
    df = df.dropna(axis=1, how="all")

    # colonnes de pas de temps = toutes sauf 'date', 'statut' (et éventuellement col_0 d'origine)
    time_cols = [c for c in df.columns if c not in ["date", "statut", "col_0"]]

    # passage en long : (date, statut, slot_col, load_mw)
    df_long = df.melt(
        id_vars=["date", "statut"],
        value_vars=time_cols,
        var_name="slot_col",
        value_name="load_mw",
    )

    # trier
    df_long = df_long.sort_values(["date", "slot_col"]).reset_index(drop=True)

    # slot_index = rang du pas de temps dans la journée
    df_long["slot_index"] = df_long.groupby("date").cumcount()

    # hypothèse : un pas = 30 minutes
    df_long["datetime"] = df_long["date"] + pd.to_timedelta(
        df_long["slot_index"] * 30, unit="m"
    )

    # garder les colonnes utiles
    df_long["year"] = year
    df_long = df_long[["datetime", "date", "year", "statut", "slot_index", "load_mw"]]

    return df_long


def process_year(year: int):
    print(f"\n===== Traitement {year} =====")
    df_raw = load_year_raw(year)
    if df_raw is None:
        print(f"[{year}] Aucun DataFrame brut disponible, on passe.")
        return

    df_long = tidy_year(df_raw, year)

    out_path = PROCESSED_DIR / f"consommation_{year}_long.csv"
    df_long.to_csv(out_path, index=False)
    print(f"[{year}] Sauvegardé : {out_path} ({df_long.shape[0]} lignes)")


if __name__ == "__main__":
    for year in range(2010, 2025):  # 2010 à 2024 inclus
        process_year(year)



===== Traitement 2010 =====
[2010] Fichier zip introuvable : /home/onyxia/france-grid-stress-prediction/data/raw*/consommation/Historique_consommation_INST_2010.zip
[2010] Aucun DataFrame brut disponible, on passe.

===== Traitement 2011 =====
[2011] Fichier zip introuvable : /home/onyxia/france-grid-stress-prediction/data/raw*/consommation/Historique_consommation_INST_2011.zip
[2011] Aucun DataFrame brut disponible, on passe.

===== Traitement 2012 =====
[2012] Fichier zip introuvable : /home/onyxia/france-grid-stress-prediction/data/raw*/consommation/Historique_consommation_INST_2012.zip
[2012] Aucun DataFrame brut disponible, on passe.

===== Traitement 2013 =====
[2013] Fichier zip introuvable : /home/onyxia/france-grid-stress-prediction/data/raw*/consommation/Historique_consommation_INST_2013.zip
[2013] Aucun DataFrame brut disponible, on passe.

===== Traitement 2014 =====
[2014] Fichier zip introuvable : /home/onyxia/france-grid-stress-prediction/data/raw*/consommation/Historiq

In [2]:
def load_year_raw(year: int) -> pd.DataFrame | None:
    """
    Charge les données brutes pour une année :
    - si un zip existe : lit le fichier à l'intérieur (CSV ou XLS)
    - sinon, tente de lire directement le XLS/XLSX dans le dossier.
    """
    # 1) On regarde d'abord s'il y a un ZIP
    zip_path = RAW_CONS_DIR / f"Historique_consommation_INST_{year}.zip"

    if zip_path.exists():
        with zipfile.ZipFile(zip_path, "r") as z:
            inner = z.namelist()[0]
            print(f"[{year}] ZIP trouvé, contenu : {inner}")
            with z.open(inner) as f:
                if inner.lower().endswith(".csv"):
                    df_raw = pd.read_csv(f, sep=";", engine="python")
                elif inner.lower().endswith((".xls", ".xlsx")):
                    df_raw = pd.read_excel(f)
                else:
                    print(f"[{year}] Format non supporté dans le zip : {inner}")
                    return None
        return df_raw

    # 2) Sinon, on cherche un XLS/XLSX directement
    xls_path = RAW_CONS_DIR / f"Historique_consommation_INST_{year}.xls"
    xlsx_path = RAW_CONS_DIR / f"Historique_consommation_INST_{year}.xlsx"

    if xls_path.exists():
        print(f"[{year}] Pas de zip, lecture directe du XLS : {xls_path.name}")
        return pd.read_excel(xls_path)

    if xlsx_path.exists():
        print(f"[{year}] Pas de zip, lecture directe du XLSX : {xlsx_path.name}")
        return pd.read_excel(xlsx_path)

    print(f"[{year}] Aucun fichier trouvé (ni zip, ni xls/xlsx).")
    return None


In [ ]:
process_year(2024)



===== Traitement 2024 =====
[2024] Pas de zip, lecture directe du XLS : Historique_consommation_INST_2024.xls
[2024] Sauvegardé : /home/onyxia/france-grid-stress-prediction/data/processed/consommation_2024_long.csv (17568 lignes)


In [4]:
process_year(2020)



===== Traitement 2020 =====
[2020] ZIP trouvé, contenu : Historique_consommation_INST_2020.xls
[2020] Sauvegardé : /home/onyxia/france-grid-stress-prediction/data/processed/consommation_2020_long.csv (7248 lignes)
